In [10]:
import tensorflow as tf

In [11]:
from tensorflow.keras.applications.resnet50 import ResNet50

In [12]:
from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

In [13]:
# 케라스 데이터셋을 다운받아 변수에 각각 넣어준다.
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

170498071/170498071 [==============================] - 3s 0us/step


In [14]:
train_images.shape, train_labels.shape, test_images.shape, test_labels.shape

((50000, 32, 32, 3), (50000, 1), (10000, 32, 32, 3), (10000, 1))

In [15]:
# sparse categorical crossentropy VS categorical crossentropy + one_hot
y_train = tf.keras.utils.to_categorical(train_labels, 10)
y_test = tf.keras.utils.to_categorical(test_labels, 10)
y_train.shape, y_test.shape

((50000, 10), (10000, 10))

In [16]:
# 라벨 설정
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

### build cnn_model

In [17]:
def cnn_model(n_hidden_node, dropout_prob):
    base_model = ResNet50(include_top=False, input_shape = (32,32 ,3), weights = 'imagenet')
    base_model.trainable = False

    inputs = tf.keras.Input(shape=(32, 32, 3))

    x = base_model(inputs, training=False)
    x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
    x = tf.keras.layers.Dense(n_hidden_node, activation='relu')(x)
    x= tf.keras.layers.Dropout(dropout_prob)(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)

    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate= 0.001),
                  loss = 'categorical_crossentropy',
                  metrics=['accuracy'])
    return model

### train five cnn models

In [18]:
cnn_v1 = cnn_model(1024, 0.5)
cnn_v2 = cnn_model(1024, 0.6)
cnn_v3 = cnn_model(1024, 0.7)
cnn_v4 = cnn_model(1024, 0.8)
cnn_v5 = cnn_model(1024, 0.9)

In [19]:
cnn_v1.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 290s 722ms/step - loss: 1.5093 - accuracy: 0.5104 - val_loss: 1.1601 - val_accuracy: 0.5928


In [20]:
cnn_v2.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 293s 736ms/step - loss: 1.6016 - accuracy: 0.4845 - val_loss: 1.1811 - val_accuracy: 0.5841


In [21]:
cnn_v3.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 278s 701ms/step - loss: 1.6931 - accuracy: 0.4592 - val_loss: 1.2239 - val_accuracy: 0.5681


In [22]:
cnn_v4.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 239s 604ms/step - loss: 1.8735 - accuracy: 0.4105 - val_loss: 1.2761 - val_accuracy: 0.5553


In [23]:
cnn_v5.fit(train_images, y_train, epochs = 1, validation_data=(test_images, y_test), batch_size=128)

391/391 [==============================] - 239s 601ms/step - loss: 2.2737 - accuracy: 0.3108 - val_loss: 1.4299 - val_accuracy: 0.5168


### evaluate five cnn model

In [24]:
pred_v1 = cnn_v1.predict(test_images)
pred_v2 = cnn_v2.predict(test_images)
pred_v3 = cnn_v3.predict(test_images)
pred_v4 = cnn_v4.predict(test_images)
pred_v5 = cnn_v5.predict(test_images)

313/313 [==============================] - 45s 139ms/step


In [25]:
import numpy as np
from sklearn.metrics import accuracy_score

y_test = np.argmax(y_test, axis=1)

score_v1 = accuracy_score(np.argmax(pred_v1, axis=1), y_test)
score_v2 = accuracy_score(np.argmax(pred_v2, axis=1), y_test)
score_v3 = accuracy_score(np.argmax(pred_v3, axis=1), y_test)
score_v4 = accuracy_score(np.argmax(pred_v4, axis=1), y_test)
score_v5 = accuracy_score(np.argmax(pred_v5, axis=1), y_test)

print(f'score_v1 : {score_v1}')
print(f'score_v2 : {score_v2}')
print(f'score_v3 : {score_v3}')
print(f'score_v4 : {score_v4}')
print(f'score_v5 : {score_v5}')

score_v1 : 0.5928
score_v2 : 0.5841
score_v3 : 0.5681
score_v4 : 0.5553
score_v5 : 0.5168


### ensemble 5 cnn model

In [31]:
pred_ensemble = np.mean([pred_v1, pred_v2, pred_v3, pred_v4, pred_v5], axis=0)

In [32]:
pred_ensemble = np.argmax(pred_ensemble, axis=1)
accuracy_score(pred_ensemble, y_test)

0.5993

### (practice)ensemble + 2 cnn model
위 다섯개의 모델을 앙상블한 모델에 아래 조건에 맞는 2개의 모델을 추가로 학습하여 앙상블해봅시다.
- n_hidden_node : 512, dropout_prob : 0.5
- n_hidden_node : 256, dropout_prob : 0.5